https://learning.oreilly.com/library/view/deep-learning-with/9781838823412/Text/Chapter_06.xhtml#_idParaDest-175

In [1]:
import tensorflow as tf
import numpy as np


https://www.tensorflow.org/datasets/keras_example

In [2]:
(X_train, _), (_,  _) = tf.keras.datasets.mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5

In [3]:
X_train = X_train.reshape(60000, 784)

In [4]:
generator = tf.keras.Sequential()
generator.add(tf.keras.layers.Dense(256, input_dim=randomDim))
generator.add(tf.keras.layers.LeakyReLU(0.2))
generator.add(tf.keras.layers.Dense(512))
generator.add(tf.keras.layers.LeakyReLU(0.2))
generator.add(tf.keras.layers.Dense(1024))
generator.add(tf.keras.layers.LeakyReLU(0.2))
generator.add(tf.keras.layers.Dense(784, activation='tanh'))

NameError: name 'randomDim' is not defined

In [5]:
discriminator = tf.keras.=Sequential()
discriminator.add(tf.keras.layers.Dense(1024, input_dim=784) )
discriminator.add(tf.keras.layers.LeakyReLU(0.2))
discriminator.add(tf.keras.layers.Dropout(0.3))
discriminator.add(tf.keras.layers.Dense(512))
discriminator.add(tf.keras.layers.LeakyReLU(0.2))
discriminator.add(tf.keras.layers.Dropout(0.3))
discriminator.add(tf.keras.layers.Dense(256))
discriminator.add(tf.keras.layers.LeakyReLU(0.2))
discriminator.add(tf.keras.layers.Dropout(0.3))
discriminator.add(tf.keras.layers.Dense(1, activation='sigmoid'))

AttributeError: module 'tensorflow.keras' has no attribute 'SequentialSequential'

In [6]:
discriminator.trainable = False
ganInput = Input(shape=(randomDim,))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = tf.keras.Model(inputs=ganInput, outputs=ganOutput)

NameError: name 'discriminator' is not defined

In [7]:
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
gan.compile(loss='binary_crossentropy', optimizer='adam')

NameError: name 'discriminator' is not defined